In [2]:
pip install pyserial pandas matplotlib


  Using cached pyserial-3.5-py2.py3-none-any.whl.metadata (1.6 kB)
Using cached pyserial-3.5-py2.py3-none-any.whl (90 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import time
import csv
import serial



In [7]:
# --------- CONFIG ----------
PORT = "COM20"  # Windows example: "COM5"
# PORT = "/dev/ttyUSB0"  # Linux example
# PORT = "/dev/tty.usbserial-XXXX"  # macOS example
BAUD = 115200

DURATION_SEC = 20  # capture length
OUT_CSV = "dual_mpu_capture.csv"

# Expected columns from your firmware:
# SEQ,T_US,A1X,A1Y,A1Z,G1X,G1Y,G1Z,A2X,A2Y,A2Z,G2X,G2Y,G2Z
EXPECTED_COLS = 14


def main():
    ser = serial.Serial(PORT, BAUD, timeout=1)
    print(f"Connected to {PORT} @ {BAUD}")

    # Give board time to reset + print header
    time.sleep(2.0)
    ser.reset_input_buffer()

    rows = 0
    bad_lines = 0
    last_seq = None
    drops = 0

    t0 = time.time()

    with open(OUT_CSV, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(
            [
                "SEQ",
                "T_US",
                "A1X",
                "A1Y",
                "A1Z",
                "G1X",
                "G1Y",
                "G1Z",
                "A2X",
                "A2Y",
                "A2Z",
                "G2X",
                "G2Y",
                "G2Z",
            ]
        )

        while time.time() - t0 < DURATION_SEC:
            line = ser.readline().decode(errors="ignore").strip()
            if not line:
                continue

            # Skip header line if it appears
            if line.startswith("SEQ") or "A1X" in line:
                continue

            parts = line.split(",")
            if len(parts) != EXPECTED_COLS:
                bad_lines += 1
                continue

            try:
                seq = int(parts[0])
                # quick drop detection
                if last_seq is not None and seq != last_seq + 1:
                    drops += (seq - last_seq - 1) if seq > last_seq else 0
                last_seq = seq

                # Write as-is (string -> CSV). Plot script will parse numeric.
                w.writerow(parts)
                rows += 1

            except ValueError:
                bad_lines += 1
                continue

    ser.close()

    elapsed = time.time() - t0
    hz = rows / elapsed if elapsed > 0 else 0

    print("\n--- Capture Summary ---")
    print(f"Saved: {OUT_CSV}")
    print(f"Duration: {elapsed:.2f}s")
    print(f"Valid rows: {rows}")
    print(f"Bad lines: {bad_lines}")
    print(f"Estimated rate: {hz:.1f} Hz")
    print(f"Detected seq drops: {drops}")


if __name__ == "__main__":
    main()


Connected to COM20 @ 115200

--- Capture Summary ---
Saved: dual_mpu_capture.csv
Duration: 20.00s
Valid rows: 2000
Bad lines: 1
Estimated rate: 100.0 Hz
Detected seq drops: 0
